## Test Activations All

In [2]:
import requests

url = "http://localhost:5002/activations-all"

payload = {
    "text": "zzz",
    "model": "gpt2-small",
    "source_set": "gpt2-small-res-jb",
    "selected_layers": ["blocks.6.hook_resid_pre"],
    "secret": "secret",
    "sort_indexes": []
}

response = requests.post(url, json=payload)

# Check the response
if response.status_code == 200:
    print("Success:", response.json())
else:
    print("Failed:", response.status_code, response.text)

Success: {'error': 'Invalid request data'}


In [4]:
import torch
torch.save(response, "response_v1.pt")

In [15]:
import requests

url = "http://localhost:5002/activations-all"

payload = {
    "text": "zzzzzz",
    "model": "gemma-2-9b",
    "source_set": "res-cat",
    "selected_layers": ["23-res-cat"],
    "secret": "secret",
    "sort_indexes": []
}

response = requests.post(url, json=payload)

# Check the response
if response.status_code == 200:
    print("Success:", response.json())
else:
    print("Failed:", response.status_code, response.text)

Success: {'activations': [{'index': 67074, 'layer': '23-res-cat', 'maxValue': 358.0, 'maxValueIndex': 1, 'sumValues': 0.0, 'values': [222.0, 358.0, 0.0]}, {'index': 56918, 'layer': '23-res-cat', 'maxValue': 42.75, 'maxValueIndex': 2, 'sumValues': 0.0, 'values': [0.0, 22.625, 42.75]}, {'index': 62094, 'layer': '23-res-cat', 'maxValue': 42.0, 'maxValueIndex': 2, 'sumValues': 0.0, 'values': [0.0, 0.0, 42.0]}, {'index': 94658, 'layer': '23-res-cat', 'maxValue': 33.0, 'maxValueIndex': 1, 'sumValues': 0.0, 'values': [0.0, 33.0, 0.0]}, {'index': 68063, 'layer': '23-res-cat', 'maxValue': 30.125, 'maxValueIndex': 2, 'sumValues': 0.0, 'values': [0.0, 0.0, 30.125]}, {'index': 1209, 'layer': '23-res-cat', 'maxValue': 28.5, 'maxValueIndex': 1, 'sumValues': 0.0, 'values': [0.0, 28.5, 6.125]}, {'index': 17205, 'layer': '23-res-cat', 'maxValue': 25.25, 'maxValueIndex': 2, 'sumValues': 0.0, 'values': [0.0, 11.25, 25.25]}, {'index': 55897, 'layer': '23-res-cat', 'maxValue': 25.25, 'maxValueIndex': 1, 's

In [3]:
len(response.json()['activations'])

20

In [9]:
response.json().keys()

dict_keys(['activations', 'counts', 'tokens'])

In [4]:
import torch
torch.save(response, "response_v2.pt")

In [5]:
rv1 = torch.load("response_v1.pt")
rv2 = torch.load("response_v2.pt")

rv1 == rv2

False

In [ ]:
for activation in rv1.json()['activations'].keys():
    

In [14]:
for i in range(len(rv1.json()['activations'])):
    assert rv1.json()['activations'][i]==rv2.json()['activations'][i]
for i in range(len(rv1.json()['counts'])):
    assert rv1.json()['counts'][i]==rv2.json()['counts'][i]
for i in range(len(rv1.json()['tokens'])):
    assert rv1.json()['tokens'][i]==rv2.json()['tokens'][i]
    
assert len(rv1.json()['activations']) == len(rv2.json()['activations'])

In [9]:
rv1.json() == rv2.json()

True

In [2]:
import requests

url = "http://localhost:5002/activations-test"

payload = {
    "text": "zzz",
    "model": "gemma-2-9b",
    "layer": "23-res-cat",
    "layer_num": 23,
    "index": "5",
    "secret": "secret"
}

response = requests.post(url, json=payload)

# Check the response
if response.status_code == 200:
    print("Success:", response.json())
else:
    print("Failed:", response.status_code, response.text)

Success: {'activations': {'maxValue': 0.0, 'maxValueIndex': 0, 'values': [0.0, 0.0]}, 'tokens': ['<bos>', 'zzz']}


In [2]:
import os
from saes.saelens import SaeLensSAE
from sae_lens import HookedSAETransformer

os.environ["GEMMA_2_SAE_WEIGHTS_ROOT"] = "/root/neuronpedia-inference/neuronpedia_inference/saes/weights/"

sae, hook_name = SaeLensSAE.load(
    release="gemma-2-saes",
    sae_id="23/post_mlp_residual/131072/0_0005",
    device="cuda",
)

In [3]:
model = HookedSAETransformer.from_pretrained_no_processing("gemma-2-9b", device="cuda", dtype="bfloat16")

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Loaded pretrained model gemma-2-9b into HookedTransformer


In [7]:
import torch 

text = "zzz"
device = "cuda"

#model = HookedSAETransformer.from_pretrained_no_processing("gemma-2-9b", device="cuda", dtype="bfloat16")

sae, hook_name = SaeLensSAE.load(
    release="gemma-2-saes",
    sae_id="23/post_mlp_residual/131072/0_0005",
    device="cuda",
)

tokens = model.to_tokens(
    text, prepend_bos=model.cfg.tokenizer_prepends_bos, truncate=False
)[0]
str_tokens = model.to_str_tokens(
    text, prepend_bos=model.cfg.tokenizer_prepends_bos
)
_, cache = model.run_with_cache(tokens, stop_at_layer=24)

activation_data = cache[hook_name].to(device)

feature_activation_data = sae.encode(
    activation_data
)
act_data = torch.transpose(feature_activation_data.squeeze(0), 0, 1)



In [8]:
feature_activation_data.shape

torch.Size([1, 2, 131072])

In [34]:
cache['blocks.23.hook_resid_post'].shape

torch.Size([1, 2, 3584])

## Test Completions

In [27]:
import requests

url = "http://localhost:5002/completion"

payload = {
    'prompt': 'Hello, world!', 
    'secret': 'change_me_secret', 
    'features': [{'layer': 'blocks.9.hook_resid_pre', 'index': 21871, 'strength': 30.0}], 
    'model': 'gpt2-small', 
    'instruct': False, 
    'types': ['DEFAULT', 'STEERED'], 
    'n_completion_tokens': 50, 
    'temperature': 0.7, 
    'strength_multiplier': 1.5, 
    'freq_penalty': 0.0, 
    'seed': 74
}

response = requests.post(url, json=payload)

# Check the response
if response.status_code == 200:
    print("Success:", response.json())
else:
    print("Failed:", response.status_code, response.text)

Success: {'DEFAULT': 'Hello, world! Yours truly,\n\nSebastian\n\nVitality\n\nFascinating\n\nInteresting\n\nInteresting\n\nInteresting\n\nInteresting\n\nInteresting\n\nInteresting\n\nInteresting\n\n', 'STEERED': "Hello, world!\n\nWe are a team of believers. Who are our fellow Saints, who are our friends, who are our friends? And who are our friends who are living God's plan? Who will? Who will die for His Command? Who will be"}


In [40]:
import requests

url = "http://localhost:5002/completion"

payload = {
    'prompt': 'Hello, Gemma! What can you teach me to do?', 
    'secret': 'change_me_secret', 
    'features': [{'layer': 'blocks.12.hook_resid_post', 'index': 13132, 'strength': 30.0}], 
    'model': 'gemma-2b-it', 
    'instruct': True, 
    'types': ['DEFAULT', 'STEERED'], 
    'n_completion_tokens': 50, 
    'temperature': 0.7, 
    'strength_multiplier': 2.5, 
    'freq_penalty': 0.0, 
    'seed': 42
}

response = requests.post(url, json=payload)

# Check the response
if response.status_code == 200:
    print("Success:", response.json())
else:
    print("Failed:", response.status_code, response.text)

Success: {'DEFAULT': "<bos><start_of_turn>user\nHello, Gemma! What can you teach me to do?<end_of_turn>\n<start_of_turn>model\nHello! I'd love to help you learn and grow. What would you like to learn more about today?\n\nI can help with a variety of topics, including:\n\n* **Language learning:** I can provide you with language lessons in English", 'STEERED': "<bos><start_of_turn>user\nHello, Gemma! What can you teach me to do?<end_of_turn>\n<start_of_turn>model\npowering on! I'd love to help you learn some new things. What would you like to learn?\n\nHere are some examples of the things I can help you with:\n\n* **Basic skills:** learning to use a computer, reading"}
